In [22]:
import pandas as pd
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import ast
from surprise import Dataset, Reader, KNNBasic
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split# as skl_train_test_split
from datetime import datetime
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
from surprise import accuracy



nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\helen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\helen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\helen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
#Data

df_business = pd.read_csv('yelp_academic_dataset_business.csv')
df_checkin = pd.read_csv('yelp_academic_dataset_checkin.csv')
df_review=pd.read_csv('yelp_academic_dataset_review.csv')
df_tip=pd.read_csv('yelp_academic_dataset_tip.csv')
df_user=pd.read_csv('yelp_academic_dataset_user.csv')

# print(df_business.columns)
# print(df_checkin.columns)
# print(df_review.columns)
# print(df_tip.columns)
# print(df_user.columns)

C:\Users\helen\AppData\Local\Temp\ipykernel_35232\1838509011.py:7: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_user=pd.read_csv('yelp_academic_dataset_user.csv')


In [3]:
#Subset de dados

df_business_filadelfia = df_business[(df_business['city'] == 'Philadelphia') & (df_business['categories'].str.contains('Restaurants', na=False)) & (df_business['is_open']==1)].reset_index(drop=True)
df_business_filadelfia=df_business_filadelfia[['business_id', 'name', 'stars', 'review_count', 'attributes', 'categories', 'hours']].reset_index(drop=True)

df_review_filadelfia = df_review[df_review['business_id'].isin(df_business_filadelfia['business_id'])]
df_review_filadelfia=df_review_filadelfia[['review_id', 'user_id', 'business_id', 'stars', 'text', 'date']]
df_review_filadelfia['liked'] = (df_review_filadelfia['stars'] > 3).astype(int)
df_review_filadelfia_profiles = df_review_filadelfia[df_review_filadelfia['liked'] == 1].reset_index(drop=True)

df_user_filadelfia = df_user[df_user['user_id'].isin(df_review_filadelfia['user_id'])]
df_user_filadelfia=df_user_filadelfia[['user_id', 'name', 'review_count', 'yelping_since', 'elite', 'average_stars']].reset_index(drop=True)

# Perfis LDA
user_profile = pd.read_csv("user_profiles.csv")
user_profile = user_profile.drop(columns=['Unnamed: 0'])
restaurant_profile = pd.read_csv("restaurant_profiles.csv")
restaurant_profile = restaurant_profile.drop(columns=['Unnamed: 0'])

In [379]:
df_review_filadelfia

,review_id,user_id,business_id,stars,text,date,liked
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,1
16,oyaMhzBSwfGgemSGuZCdwQ,Dd1jQj7S-BFGqRbApFzCFw,YtSqYv1Q_pOltsVPSx54SA,5.0,Tremendous service (Big shout out to Douglas) ...,2013-06-24 11:21:25,1
19,Xs8Z8lmKkosqW5mw_sVAoA,IQsF3Rc6IgCzjVV9DE8KXg,eFvzHawVJofxSnD7TgbZtg,5.0,My absolute favorite cafe in the city. Their b...,2014-11-12 15:30:27,1
38,JBWZmBy69VMggxj3eYn17Q,aFa96pz67TwOFu4Weq5Agg,kq5Ghhh14r-eCxlVmlyd8w,5.0,My boyfriend and I tried this deli for the fir...,2018-08-23 21:39:38,1
42,YcLXh-3UC9y6YFAI9xxzPQ,G0DHgkSsDozqUPWtlxVEMw,oBhJuukGRqPVvYBfTkhuZA,4.0,The only reason I didn't give this restaurant ...,2015-03-05 03:37:54,1
...,...,...,...,...,...,...,...
6990235,ZcEx4UEnTnR_TEPEqwkKjA,gkg9VqsxPCgpfYXO1dl8CA,Ea663rIHyKXz2VP2DPH7Cg,4.0,I decided to try this place out after Christma...,2020-01-13 04:21:38,1
6990239,me7QTotYCOjWNVA8bzN1eg,bJ5FtCtZX3ZZacz2_2PJjA,wMQkdK2aNMvq2xoojC98Mw,4.0,South Street Diner isn't the best of Philly Di...,2007-07-27 20:12:11,1
6990240,5n_oSwXspiiSsZgNwjp48g,bJ5FtCtZX3ZZacz2_2PJjA,SOsjW1JARmtHUFtpFlp8rw,4.0,When I first heard that the Peace A Pizza (htt...,2017-02-23 19:11:04,1
6990255,MVg4YUQeEhCA7Z7RsBJSVg,7-7A0Avj47slLGV7yBFc8w,ytynqOUb3hjKeJfRj5Tshw,3.0,"I was so excited about all the food I saw, but...",2013-07-25 21:00:15,0


In [380]:
# counts
user_counts = df_review_filadelfia['user_id'].value_counts()
restaurant_counts = df_review_filadelfia['business_id'].value_counts()

# creating filters for users and restaurants with 5+ reviews
users_with_5_plus_reviews = user_counts[user_counts >= 5].index
restaurants_with_5_plus_reviews = restaurant_counts[restaurant_counts >= 5].index

In [381]:
print(f'Number of users with 5+ reviews: {len(users_with_5_plus_reviews)}')
print(f'Number of restaurants reviewed 5+ times: {len(restaurants_with_5_plus_reviews)}')

Number of users with 5+ reviews: 13948
Number of restaurants reviewed 5+ times: 3040


In [402]:
# Desired dataset shape
#data = df_review_filadelfia_profiles[['user_id', 'business_id', 'text', 'stars']] # only positive reviews
data = df_review_filadelfia[['user_id', 'business_id', 'text', 'stars']]

data_sample = data.sample(100000, random_state=10).reset_index(drop=True)

# train-test split
X_train, X_test, y_train, y_test = skl_train_test_split(data_sample[['user_id', 'business_id', 'text']], data_sample['stars'], test_size=0.2, random_state=1)
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

# this outputs the topic matrix according to the method chosen (bag-of-word, lsa, lda and doc2vec) 
n_components = 8
topic_matrix = feature_engineering(X_train, 'lda', n_components)

# attach topic matrix to the dataset
column_names = ['comp_{}'.format(i+1) for i in range(n_components)]
topics = pd.DataFrame(topic_matrix, columns=column_names)

# create profiles
user_profile = pd.concat([X_train, topics], axis=1).drop(columns=['business_id', 'text']).groupby("user_id", as_index=False)[column_names].mean()
restaurant_profile = pd.concat([X_train, topics], axis=1).drop(columns=['user_id', 'text']).groupby("business_id", as_index=False)[column_names].mean()

# filtering for the ones with 5+ reviews (more representative)
user_profile = user_profile[user_profile['user_id'].isin(users_with_5_plus_reviews)].reset_index(drop=True)
restaurant_profile = restaurant_profile[restaurant_profile['business_id'].isin(restaurants_with_5_plus_reviews)].reset_index(drop=True)

# predicting on the test set using the recommendations function
X_test['star_pred'] = X_test.apply(lambda row: recommend(row['user_id'], row['business_id'], df_review_filadelfia, user_profile, restaurant_profile, type='users'), axis=1)
y = pd.concat([X_test['star_pred'], y_test], axis=1).dropna()

# metrics
np.sqrt(mean_squared_error(y['star_pred'], y['stars']))

1.0464826958020617

In [403]:
# função para dar input de user e business e output de rating (recomendações)
X_test['star_pred'] = X_test.apply(lambda row: recommend(row['user_id'], row['business_id'], df_review_filadelfia, user_profile, restaurant_profile, type='items'), axis=1)
y = pd.concat([X_test['star_pred'], y_test], axis=1).dropna()

# avaliar no rating ground truth
np.sqrt(mean_squared_error(y['star_pred'], y['stars']))

1.273092206852342

In [405]:
# função para dar input de user e business e output de rating (recomendações)
X_test['star_pred'] = X_test.apply(lambda row: recommend(row['user_id'], row['business_id'], df_review_filadelfia, user_profile, restaurant_profile, type='user_item'), axis=1)
y = pd.concat([X_test['star_pred'], y_test], axis=1).dropna()

# avaliar no rating ground truth
np.sqrt(mean_squared_error(y['star_pred'], y['stars']))

1.1661959908374837

In [406]:
# função para dar input de user e business e output de rating (recomendações)
X_test['star_pred'] = X_test.apply(lambda row: recommend(row['user_id'], row['business_id'], df_review_filadelfia, user_profile, restaurant_profile, type='users'), axis=1)
y = pd.concat([X_test['star_pred'], y_test], axis=1).dropna()

# avaliar no rating ground truth
np.sqrt(mean_squared_error(y['star_pred'], y['stars']))

1.0464826958020617

In [20]:
def map_rating(cosine_similarity):
    '''
    Maps a consine similarity score to a rating from 1 to 5
    '''
    return 1 + 4 * ((cosine_similarity + 1) / 2)

In [3]:
#Load data

def load_data():
    # df_business = pd.read_csv('yelp_academic_dataset_business.csv')
    # df_review=pd.read_csv('yelp_academic_dataset_review.csv')
    # df_user=pd.read_csv('yelp_academic_dataset_user.csv')
    
    df_business_filadelfia = df_business[(df_business['city'] == 'Philadelphia') & (df_business['categories'].str.contains('Restaurants', na=False)) & (df_business['is_open']==1)].reset_index(drop=True)
    df_business_filadelfia=df_business_filadelfia[['business_id', 'name', 'stars', 'review_count', 'attributes', 'categories', 'hours']].reset_index(drop=True)
    df_business_filadelfia = df_business_filadelfia.dropna().reset_index(drop=True)

    df_review_filadelfia = df_review[df_review['business_id'].isin(df_business_filadelfia['business_id'])]
    df_review_filadelfia=df_review_filadelfia[['review_id', 'user_id', 'business_id', 'stars', 'text', 'date']]
    df_review_filadelfia['liked'] = (df_review_filadelfia['stars'] > 3).astype(int)
    df_review_filadelfia_profiles = df_review_filadelfia[df_review_filadelfia['liked'] == 1].reset_index(drop=True)
    df_review_filadelfia = df_review_filadelfia.dropna().reset_index(drop=True)

    df_user_filadelfia = df_user[df_user['user_id'].isin(df_review_filadelfia['user_id'])]
    df_user_filadelfia=df_user_filadelfia[['user_id', 'name', 'review_count', 'yelping_since', 'elite', 'average_stars']].reset_index(drop=True)
    df_user_filadelfia = df_user_filadelfia.dropna().reset_index(drop=True)

    # counts
    user_counts = df_review_filadelfia['user_id'].value_counts()
    restaurant_counts = df_review_filadelfia['business_id'].value_counts()

    # creating filters for users and restaurants with 5+ reviews
    users_with_5_plus_reviews = user_counts[user_counts >= 5].index
    restaurants_with_5_plus_reviews = restaurant_counts[restaurant_counts >= 5].index

    return df_business_filadelfia,df_review_filadelfia,df_user_filadelfia, users_with_5_plus_reviews, restaurants_with_5_plus_reviews

In [4]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(lemmatized_words)

In [5]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def stem_text(text):
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words if word not in stop_words]
    return ' '.join(stemmed_words)


In [6]:
#Pre-processamento
def pre_processing(data_review,method):
    if method == 'with lemma':

        data_review['text'] = data_review['text'].apply(lemmatize_text)

    
    else: 
        data_review['text'] = data_review['text'].apply(stem_text)

    return data_review


In [22]:
test_data = df_review[:100]
test_data

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...,...,...,...,...,...
95,QS7CuOtFLuS3dnwKHRtSYQ,PBnEwGVCBL0N-bET6ZI6kQ,m5-FtgWRd4qA7j0vaOXiXQ,5.0,0,0,0,Had to wait until my third trip to NOLA to act...,2016-11-10 20:56:13
96,4PHFo_GRG4FEk1q4X7xQVQ,jbsCBG0A-3wVDjrKar-0Wg,X63jIMRHYBvKKQDuJTRiQQ,5.0,2,0,1,A GREAT EXPERIENCE!!!!!!!!! I was a completel...,2014-10-11 13:55:05
97,1c6sgLe07HAhipebsQ1wgA,ZRXvbrutBBULaFS6T9NCwA,HnhxO2cpa15AHI1456Pl6Q,5.0,0,1,0,Wow! I never thought my sons phone could be re...,2015-10-17 00:55:35
98,PPgbLBvi34A6m7bKJfTwhw,3TL6HZ1JrKcNTvGDWKlrow,GyC36Pn0Q1-qHnqXys6yFg,1.0,0,0,0,Service and management terrible... After messi...,2013-12-07 13:17:13


In [7]:
#Feature Engineering
def feature_engineering(data_reviews, method, components=8):
    if method == 'bag of words':
        vectorizer = CountVectorizer(max_df=0.95, min_df=2)
        matrix = vectorizer.fit_transform(data_reviews['text'])
        matrix = matrix.toarray()
        components = matrix.shape[1]
        # feature_names = vectorizer.get_feature_names_out()
        # df = pd.DataFrame(matrix, columns=feature_names)
   
    elif method =='lda':
        count_vectorizer = CountVectorizer(max_df=0.95, min_df=2)
        count_matrix = count_vectorizer.fit_transform(data_reviews['text'])
        lda_model = LatentDirichletAllocation(n_components=components, random_state=42)
        matrix = lda_model.fit_transform(count_matrix)
        # df = pd.DataFrame(matrix, columns=[f'Topic_{i}' for i in range(lda_model.n_components)])
    
    elif method =='lsa':
        vectorizer = CountVectorizer()
        count_matrix = vectorizer.fit_transform(data_reviews['text'])
        lsa_model = TruncatedSVD(n_components=components)
        matrix = lsa_model.fit_transform(count_matrix)

    elif method == 'doc2vec':
        # preproces the documents, and create TaggedDocuments
        tagged_data = [TaggedDocument(words=word_tokenize(doc.lower()),
                                    tags=[str(i)]) for i,
                    doc in enumerate(data_reviews['text'])]

        # Doc2vec model
        model = Doc2Vec(vector_size=components,
                        min_count=2, epochs=50)
        model.build_vocab(tagged_data)
        model.train(tagged_data,
                    total_examples=model.corpus_count,
                    epochs=model.epochs)

        # document vectors
        matrix = [model.infer_vector(
            word_tokenize(doc.lower())) for doc in data_reviews['text']]

    return matrix, components


In [8]:
def features_business(df_business_filadelfia):
    df_business_filadelfia = df_business_filadelfia[['business_id', 'name', 'stars', 'review_count','attributes', 'categories', 'hours']]


    #variavel horario
    df_business_filadelfia['hours'] = df_business_filadelfia['hours'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else {})

    # Crie colunas separadas para cada dia da semana
    dias_da_semana = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    for dia in dias_da_semana:
        df_business_filadelfia[dia] = df_business_filadelfia['hours'].apply(lambda x: x.get(dia, None))

    # Remova a coluna original 'hours' se não for mais necessária
    df_business_filadelfia.drop(columns=['hours'], inplace=True)


    def categorize_hours(hours):

        if hours == None:
            return 0 #'Fechado'
        
        else:
            start_time, end_time = hours.split('-')
            start_hour = int(start_time.split(':')[0])
            end_hour = int(end_time.split(':')[0])
            
            if end_hour <= 12:
                return 1 #'Manhã'
            elif 12 < start_hour and end_hour<=15:
                return 2 #'Almoço'
            elif start_hour > 15 and end_hour < 19:
                return 3 #'Tarde'
            elif start_hour>=19:
                return 4 #'Noite'
            else:
                return 5 #'Dia todo'

    # Aplicar a função de categorização a cada coluna de dia da semana
    for day in ['Monday', 'Tuesday', 'Wednesday','Thursday', 'Friday', 'Saturday', 'Sunday']:
        df_business_filadelfia[day] = df_business_filadelfia[day].apply(categorize_hours)


    #variavel categoria

    df_business_filadelfia['Restaurants'] = 0
    df_business_filadelfia['Food'] = 0
    df_business_filadelfia['Nightlife'] = 0
    df_business_filadelfia['Bars'] = 0
    df_business_filadelfia['Sandwiches'] = 0
    df_business_filadelfia['American (New)'] = 0
    df_business_filadelfia['Pizza'] = 0
    df_business_filadelfia['Breakfast & Brunch'] = 0
    df_business_filadelfia['American (Traditional)'] = 0
    df_business_filadelfia['Coffee & Tea'] = 0

    for index, row in df_business_filadelfia.iterrows():
        categories = row['categories']
        if 'Restaurants' in categories:
            df_business_filadelfia.at[index, 'Restaurantes'] = 1

        if 'Food' in categories:
            df_business_filadelfia.at[index, 'Food'] = 1

        if 'Nightlife' in categories:
            df_business_filadelfia.at[index, 'Nightlife'] = 1

        if 'Bars' in categories:
            df_business_filadelfia.at[index, 'Bars'] = 1

        if 'Sandwiches' in categories:
            df_business_filadelfia.at[index, 'Sandwiches'] = 1

        if 'American (New)' in categories:
            df_business_filadelfia.at[index, 'American (New)'] = 1
        
        if 'Pizza' in categories:
            df_business_filadelfia.at[index, 'Pizza'] = 1

        if 'Breakfast & Brunch' in categories:
            df_business_filadelfia.at[index, 'Breakfast & Brunch'] = 1

        if 'American (Traditional)' in categories:
            df_business_filadelfia.at[index, 'American (Traditional)'] = 1

        if 'Coffee & Tea' in categories:
            df_business_filadelfia.at[index, 'Coffee & Tea'] = 1



        #variavel atributos

        df_business_filadelfia['RestaurantsDelivery'] = 0
        df_business_filadelfia['OutdoorSeating'] = 0
        df_business_filadelfia['BusinessAcceptsCreditCards'] = 0
        df_business_filadelfia['BikeParking'] = 0
        df_business_filadelfia['RestaurantsTakeOut'] = 0
        df_business_filadelfia['ByAppointmentOnly'] = 0
        df_business_filadelfia['Caters'] = 0
        df_business_filadelfia['RestaurantsReservations'] = 0
        df_business_filadelfia['RestaurantsGoodForGroups'] = 0
        df_business_filadelfia['HasTV'] = 0
        df_business_filadelfia['GoodForKids'] = 0
        df_business_filadelfia['DogsAllowed'] = 0
        df_business_filadelfia['HappyHour'] = 0
        df_business_filadelfia['WheelchairAccessible'] = 0
        df_business_filadelfia['RestaurantsTableService'] = 0
        df_business_filadelfia['BusinessAcceptsBitcoin'] = 0
        df_business_filadelfia['Corkage'] = 0
        df_business_filadelfia['CoatCheck'] = 0
        df_business_filadelfia['BYOB'] = 0
        df_business_filadelfia['DriveThru'] = 0


        for index, row in df_business_filadelfia.iterrows():
            categories = row['categories']
            if 'RestaurantsDelivery' in categories:
                df_business_filadelfia.at[index, 'RestaurantsDelivery'] = 1

            if 'OutdoorSeating' in categories:
                df_business_filadelfia.at[index, 'OutdoorSeating'] = 1

            if 'BusinessAcceptsCreditCards' in categories:
                df_business_filadelfia.at[index, 'BusinessAcceptsCreditCards'] = 1

            if 'BikeParking' in categories:
                df_business_filadelfia.at[index, 'BikeParking'] = 1

            if 'RestaurantsTakeOut' in categories:
                df_business_filadelfia.at[index, 'RestaurantsTakeOut'] = 1

            if 'ByAppointmentOnly' in categories:
                df_business_filadelfia.at[index, 'ByAppointmentOnly'] = 1
            
            if 'Caters' in categories:
                df_business_filadelfia.at[index, 'Caters'] = 1

            if 'RestaurantsReservations' in categories:
                df_business_filadelfia.at[index, 'RestaurantsReservations'] = 1

            if 'RestaurantsGoodForGroups' in categories:
                df_business_filadelfia.at[index, 'RestaurantsGoodForGroups'] = 1

            if 'HasTV' in categories:
                df_business_filadelfia.at[index, 'HasTV'] = 1

            if 'GoodForKids' in categories:
                df_business_filadelfia.at[index, 'GoodForKids'] = 1

            if 'DogsAllowed' in categories:
                df_business_filadelfia.at[index, 'DogsAllowed'] = 1

            if 'HappyHour' in categories:
                df_business_filadelfia.at[index, 'HappyHour'] = 1

            if 'WheelchairAccessible' in categories:
                df_business_filadelfia.at[index, 'WheelchairAccessible'] = 1

            if 'RestaurantsTableService' in categories:
                df_business_filadelfia.at[index, 'RestaurantsTableService'] = 1

            if 'BusinessAcceptsBitcoin' in categories:
                df_business_filadelfia.at[index, 'BusinessAcceptsBitcoin'] = 1

            if 'Corkage' in categories:
                df_business_filadelfia.at[index, 'Corkage'] = 1
            
            if 'CoatCheck' in categories:
                df_business_filadelfia.at[index, 'CoatCheck'] = 1

            if 'BYOB' in categories:
                df_business_filadelfia.at[index, 'BYOB'] = 1

            if 'DriveThru' in categories:
                df_business_filadelfia.at[index, 'DriveThru'] = 1

        df_business_filadelfia.columns = ['business_id', 'name', 'stars', 'review_count_business', 'attributes', 'categories', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday',
       'Saturday', 'Sunday', 'Restaurants', 'Food', 'Nightlife', 'Bars', 'Sandwiches', 'American (New)', 'Pizza', 'Breakfast & Brunch', 'American (Traditional)', 'Coffee & Tea', 'Restaurantes',
       'RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'BikeParking', 'RestaurantsTakeOut', 'ByAppointmentOnly', 'Caters','RestaurantsReservations', 'RestaurantsGoodForGroups', 'HasTV',
       'GoodForKids', 'DogsAllowed', 'HappyHour', 'WheelchairAccessible','RestaurantsTableService', 'BusinessAcceptsBitcoin', 'Corkage','CoatCheck', 'BYOB', 'DriveThru']
        
        df_business_filadelfia=df_business_filadelfia.drop(['attributes', 'categories','name'],axis=1)

        return df_business_filadelfia

In [9]:
def features_user(df_user_filadelfia):


    yelping_since = pd.to_datetime(df_user_filadelfia['yelping_since'])

    # Calcule o número de anos no Yelp
    current_year = datetime.now().year
    df_user_filadelfia['yelping_since'] = current_year - yelping_since.dt.year


    df_user_filadelfia.columns = ['user_id', 'name', 'review_count_user', 'yelping_years', 'elite', 'average_stars']
    df_user_filadelfia=df_user_filadelfia.drop(['elite','name'],axis=1)
    # df_user_filadelfia=df_user_filadelfia.drop('yelping_since',axis=1)

        

    return df_user_filadelfia

In [10]:
def profiling(X_train, method, n_components, users_with_5_plus_reviews, restaurants_with_5_plus_reviews):
    
    # this outputs the topic matrix according to the method chosen (bag-of-word, lsa, lda and doc2vec) 
    topic_matrix, n_components = feature_engineering(X_train, method, n_components)

    # attach topic matrix to the dataset
    column_names = ['comp_{}'.format(i+1) for i in range(n_components)]
    topics = pd.DataFrame(topic_matrix, columns=column_names)

    # create profiles
    user_profile = pd.concat([X_train, topics], axis=1).drop(columns=['business_id', 'text']).groupby("user_id", as_index=False)[column_names].mean()
    restaurant_profile = pd.concat([X_train, topics], axis=1).drop(columns=['user_id', 'text']).groupby("business_id", as_index=False)[column_names].mean()

    # filtering for the ones with 5+ reviews (more representative)
    user_profile = user_profile[user_profile['user_id'].isin(users_with_5_plus_reviews)].reset_index(drop=True)
    restaurant_profile = restaurant_profile[restaurant_profile['business_id'].isin(restaurants_with_5_plus_reviews)].reset_index(drop=True)

    return user_profile, restaurant_profile

In [23]:
def split(df_review_filadelfia):
    # Desired dataset shape
    #data = df_review_filadelfia_profiles[['user_id', 'business_id', 'text', 'stars']] # only positive reviews
    data = df_review_filadelfia[['user_id', 'business_id', 'text', 'stars']]

    #data_sample = data.sample(100000, random_state=10).reset_index(drop=True)

    # train-test split
    X_train, X_test, y_train, y_test = train_test_split(data[['user_id', 'business_id', 'text']], data['stars'], test_size=0.2, random_state=1)
    X_train.reset_index(drop=True, inplace=True)
    X_test.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)

    return X_train, X_test, y_train, y_test

In [12]:
# Recommendation system

def recommend(user_id, restaurant_id, df_review_filadelfia, user_profile, restaurant_profile, type):
    
    '''
    Estimates rating a user gives to a restaurant

    Inputs:
    user_id - the user
    restaurant_id - the restaurant to be rated
    df_review_filadelfia - original df with no filtering regarding the review being positive or not
    user_profile - df with the profiles of the users (vectors from LSA/LDA/doc2vec)
    restaurant_profile - df with the profiles of the restaurants (vectors from LSA/LDA/doc2vec)
    type - type of recommendations (user_item, users or items) (default = "user_item")

    Outputs:
    Rating
    '''

    try:

        # Extracting the vector relative to the user and removing the user from the profiles
        usr_lst = user_profile[user_profile['user_id'] == user_id].drop(columns=['user_id']).fillna(0).values
        user_profile_function = user_profile[user_profile['user_id'] != user_id].reset_index(drop=True).fillna(0)
        
        # Extracting the vector relative to the restaurant and removing the restaurant from the profiles
        bus_lst = restaurant_profile[restaurant_profile['business_id'] == restaurant_id].drop(columns=['business_id']).fillna(0).values
        restaurant_profile_function = restaurant_profile[restaurant_profile['business_id'] != restaurant_id].reset_index(drop=True).fillna(0)


        if type == 'UIBH':
            # Measures the similarity between user and restaurant
            # Rating is a linear function of the similarity

            # Removing the added feature so that the vectors have the same dimensions
            usr_lst = usr_lst[:,:-3]
            bus_lst = bus_lst[:,:-40]

            similarity_score = cosine_similarity(usr_lst, bus_lst)
            rating = map_rating(similarity_score[0][0])
            
        elif type == 'UBH':
            # Measures the similarity between the user and the other users that rated the restaurant
            # Rating is a weighted average of the ratings given by the users (weighted by the similarity)

            # Getting the other users that rated the restaurant and removing user
            users = df_review_filadelfia[df_review_filadelfia['business_id'] == restaurant_id]['user_id']
            users = users[users != user_id]

            # Due to considering only the positive reviews for the profiles, some users don't have profile
            # This is to remove them from the users list
            users = users[users.isin(user_profile_function['user_id'])].unique() # and remove duplicates

            # Getting the ratings given by the users and averaging if there are more than one
            users_ratings = df_review_filadelfia[df_review_filadelfia['user_id'].isin(users)][['user_id', 'stars']]
            users_ratings = users_ratings.groupby('user_id').mean().reset_index()

            # Creating a matrix for the similar users
            user_matrix = user_profile_function[user_profile_function['user_id'].isin(users)].drop(columns=['user_id']).values

            # Similarities between user and users
            similarity_scores = cosine_similarity(usr_lst, user_matrix)

            # Transform similarities into weights
            # This assumes that there will be other similar users.
            # If all the other users are not similar we are giving high weights to "not similar" users due to this rescaling
            similarity_scores = (similarity_scores+1)/2
            weights = similarity_scores / np.sum(similarity_scores, axis=1)

            # Computing weight-averaged rating
            rating = np.dot(weights[0], users_ratings['stars'])

        elif type == 'IBH':
            # Measures the similarity between the restaurant and other 
            # Recommends the restaurants that are most similar to the ones the user liked before

            # Getting the other restaurants the user rated and removing the restaurant
            restaurants = df_review_filadelfia[df_review_filadelfia['user_id'] == user_id]['business_id']
            restaurants = restaurants[restaurants != restaurant_id]

            # Due to considering only the positive reviews for the profiles, some restaurants don't have profile
            # This is to remove them from the restaurants list
            restaurants = restaurants[restaurants.isin(restaurant_profile_function['business_id'])].unique() # and remove duplicates

            # Getting the ratings given by the user and averaging if there are more than one
            user_ratings = df_review_filadelfia[df_review_filadelfia['user_id'] == user_id][['business_id', 'stars']]
            user_ratings = user_ratings[user_ratings['business_id'].isin(restaurants)].reset_index(drop=True)
            user_ratings = user_ratings.groupby('business_id').mean().reset_index()

            # Creating a matrix for the similar restaurants
            restaurant_matrix = restaurant_profile_function[restaurant_profile_function['business_id'].isin(restaurants)].drop(columns=['business_id']).fillna(0).values

            # Similarities between the restaurant and the other restaurants
            similarity_scores = cosine_similarity(bus_lst, restaurant_matrix)

            # Transform similarities into weights
            # This assumes that there will be other similar restaurants.
            # If all the other restaurants are not similar we are giving high weights to "not similar" restaurants due to this rescaling
            similarity_scores = (similarity_scores+1)/2
            weights = similarity_scores / np.sum(similarity_scores, axis=1)

            # Computing weight-averaged rating
            rating = np.dot(weights[0], user_ratings['stars'])
        
        else:
            raise ValueError("Invalid type. Please choose 'UIBH', 'UBH', or 'IBH'.")
        
        return rating
    
    except:
        return np.nan

In [14]:
def precision_at_k(ground_truth, recommendations, k):

    #top k predicted restaurants
    top_k_pred = recommendations[:k]
    
    #top k user preferred restaurants
    top_k_true = ground_truth[:k]
    
    # number of relevant items in the top k predictions
    relevant = len(set(top_k_pred) & set(top_k_true))
    
    return relevant / k


def recall_at_k(ground_truth, recommendations, k):

    # top k predicted restaurants
    top_k_pred = recommendations[:k]
    
    # total number of preferred restaurants
    total_relevant = len(ground_truth)
    
    #number of relevant items in the top k predictions
    relevant = len(set(top_k_pred) & set(ground_truth))
    
    return relevant / total_relevant if total_relevant > 0 else 0

# TEMOS QUE APLICAR ISTO A CADA USER. TEMOS QUE TER UMA FUNÇÃO QUE PARA O USER DÁ TAMBÉM UM RANKING DE RESTAURANTES
# COM ESSE RANKING CALCULAR PRECISION E RECALL @ K
# E DEPOIS CALCULAR A MÉDIA DE TODOS OS USERS PARA ESSE MÉTODO

# OU ENTÃO AGRUPAR AS PEDICTIONS POR USER E ASSUMIR ESSE RANKING
# MARTELAR PARA SEREM SÓ USERS COM MAIS DE K REVIEWS

In [27]:
def test_evaluate(X_test, y_test, df_review_filadelfia, user_profile, restaurant_profile, method):
    # predicting on the test set using the recommendations function
    X_test['star_pred'] = X_test.apply(lambda row: recommend(row['user_id'], row['business_id'], df_review_filadelfia, user_profile, restaurant_profile, type=method), axis=1)
    y = pd.concat([X_test['business_id'], X_test['star_pred'], y_test], axis=1).dropna()

    # metrics
    rmse = np.sqrt(mean_squared_error(y['star_pred'], y['stars']))

    return rmse

In [26]:
# methods_pre_processing = ['with lemma','with stemma']
methods_pre_processing = ['with lemma']
# methods_feature_engineering = ['bag of words', 'word embeddings', 'lda']
# methods_feature_engineering = ['bag of words','lda', 'lsa', 'doc2vec']
methods_feature_engineering = ['lda', 'lsa', 'doc2vec']
add_features_decision = ['yes','no']
algorithms_cf = ['CF-UB','CF-IB'] #CF-IB(Colaborative Filtering Item Based),CF-UB(Colaborative Filtering User Based)
algorithms_content = ['UBH','IBH','UIBH'] #UBH(User Based Hybrid), IBH(Item Based Hybrid), UIBH(User Item Based Hybrid)

In [36]:
#Main J

def main():

    rmse_df = pd.DataFrame(columns=['Pre-processing', 'Feature Engineering', 'Algorithm', 'RMSE'])

    # Load the dataset
    df_business_filadelfia,df_review_filadelfia,df_user_filadelfia, users_with_5_plus_reviews, restaurants_with_5_plus_reviews = load_data()

    df_review_filadelfia = df_review_filadelfia.sample(1000, random_state=10).reset_index(drop=True) # TO TEST AND DELETE AFTER


    # Train-test split
    X_train, X_test, y_train, y_test = split(df_review_filadelfia)

    users_train = X_train['user_id']

    businesses_train = X_train['business_id']

    df_user_filadelfia_treino = df_user_filadelfia[df_user_filadelfia['user_id'].isin(users_train)]

    df_business_filadelfia_treino = df_business_filadelfia[df_business_filadelfia['business_id'].isin(businesses_train)]

    df_features_user = features_user(df_user_filadelfia_treino)
    df_features_business = features_business(df_business_filadelfia_treino)

    # Collaborative Filtering
    for a in algorithms_cf:
        train_data = pd.concat([X_train, y_train], axis=1)

        # Combinar X_test e y_test para formar o DataFrame de teste
        test_data = pd.concat([X_test, y_test], axis=1)

        # Definir o Reader com a escala de classificação
        reader = Reader(rating_scale=(df_review_filadelfia.stars.min(), df_review_filadelfia.stars.max()))

        train_dataset = Dataset.load_from_df(train_data[['user_id', 'business_id', 'stars']], reader)
        trainset = train_dataset.build_full_trainset()

        # Carregar os dados de teste no formato Surprise
        test_dataset = Dataset.load_from_df(test_data[['user_id', 'business_id', 'stars']], reader)
        testset = [(uid, iid, true_r) for (uid, iid, true_r) in test_data[['user_id', 'business_id', 'stars']].values]

        if a == 'CF-UB':
            algo = KNNBasic(sim_options={'user_based': True})
        else:
            algo = KNNBasic(sim_options={'user_based': False})

        algo.fit(trainset)
        predictions = algo.test(testset)
        rmse = accuracy.rmse(predictions)

        rmse_df = rmse_df.append({'Pre-processing': None, 'Feature Engineering': None, 'Algorithm': a, 'RMSE': rmse}, ignore_index=True)



    # Content-based / Hybrid
    for a in methods_pre_processing:
        X_train_pre_processed = pre_processing(X_train, a)

        for b in methods_feature_engineering:

            if b == 'doc2vec':
                n_components = 100
            else:
                n_components = 8

            user_profile, restaurant_profile = profiling(X_train_pre_processed, b, n_components, users_with_5_plus_reviews, restaurants_with_5_plus_reviews)

            user_profile=user_profile.merge(df_features_user,on='user_id')
            restaurant_profile=restaurant_profile.merge(df_features_business,on='business_id')

            for c in algorithms_content:
                # return(X_test, y_test, user_profile, restaurant_profile, df_review_filadelfia)
                rmse = test_evaluate(X_test, y_test, df_review_filadelfia, user_profile, restaurant_profile, c)
                # return rmse
                print(f'{a} - {b} - {c} - {rmse}')
                rmse_df = rmse_df.append({'Pre-processing': a, 'Feature Engineering': b, 'Algorithm': c, 'RMSE': rmse}, ignore_index=True)

    return rmse_df

In [37]:
main()

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.3390


C:\Users\helen\AppData\Local\Temp\ipykernel_35232\3323469296.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_filadelfia['yelping_since'] = current_year - yelping_since.dt.year


1.3390388343883086

In [156]:
#Main
feature_engineering_method = []
algoritmo=[]
accuracy = []
precision = []
f1_score = []
recall = []

def main():
    df_business_filadelfia,df_review_filadelfia,df_user_filadelfia = load_data()

    df_business_filadelfia=df_business_filadelfia.sample(1000)
    df_review_filadelfia=df_review_filadelfia.sample(1000)
    df_user_filadelfia=df_user_filadelfia.sample(1000)

    for a in algorithms_cf:
        if a == 'CF-UB':
            algo = KNNBasic(sim_options={'user_based': True})
            algo.fit(user_trainset)
            predictions = algo.test(user_testset)

        else:

            algo = KNNBasic(sim_options={'user_based': False})
            algo.fit(business_trainset)
            predictions = algo.test(business_testset)  


    for a in methods_pre_processing:
        df_review_filadelfia_pre_processado = pre_processing(df_review_filadelfia,a)

        for b in methods_feature_engineering:
            matrix = feature_engineering(df_review_filadelfia_pre_processado,b)

            for c in algorithms_content:
                df_review_filadelfia = df_review_filadelfia.reset_index(drop=True)

                if c == 'UBH':
                    matrix_per_user = df_review_filadelfia.groupby('user_id').apply(lambda x: np.mean(matrix[x.index], axis=0))
                    features = features_user(df_user_filadelfia)

                    matrix_df = pd.DataFrame(matrix_per_user.tolist(), index=matrix_per_user.index)

                    # Adiciona a coluna 'user_id' ao DataFrame
                    matrix_df.reset_index(inplace=True)

                    # Junta as features à matriz
                    result_df = matrix_df.merge(features, on='user_id')

                    result_matrix = result_df.set_index('user_id').to_numpy()

                    similarity_matrix = cosine_similarity(result_matrix)

                if c == 'IBH':
                    matrix_per_business = df_review_filadelfia.groupby('business_id').apply(lambda x: np.mean(matrix[x.index], axis=0))
                    features = features_business(df_business_filadelfia)

                    matrix_df = pd.DataFrame(matrix_per_business.tolist(), index=matrix_per_business.index)

                    # Adiciona a coluna 'business_id' ao DataFrame
                    matrix_df.reset_index(inplace=True)

                    # Junta as features à matriz
                    result_df = matrix_df.merge(features, on='business_id')

                    result_matrix = result_df.set_index('business_id').to_numpy()

                    #há restaurantes na matriz que não têm info nas features, mas agora já não consigo pensar

                    similarity_matrix = cosine_similarity(result_matrix)


                return similarity_matrix



In [157]:
main()


array([[1.        , 0.99544841, 0.9956561 , 0.99550571, 0.99320691,
        0.99512158, 0.97249916, 0.99370598, 0.99560994, 0.99512402,
        0.99566465, 0.99468465, 0.99510445, 0.99554181, 0.96953973,
        0.99503323, 0.9954954 ],
       [0.99544841, 1.        , 0.99861204, 0.99838974, 0.99591978,
        0.99783472, 0.97700643, 0.99572167, 0.99871058, 0.99854622,
        0.99869774, 0.99812797, 0.99797965, 0.99823546, 0.97154531,
        0.99753358, 0.99831295],
       [0.9956561 , 0.99861204, 1.        , 0.99958286, 0.99581111,
        0.99938824, 0.97197443, 0.99640836, 0.9996102 , 0.99791619,
        0.99928982, 0.99789648, 0.9978941 , 0.99941018, 0.96858762,
        0.99865675, 0.99908913],
       [0.99550571, 0.99838974, 0.99958286, 1.        , 0.99542662,
        0.99978773, 0.96997742, 0.99636747, 0.99980537, 0.99736553,
        0.99929694, 0.99763979, 0.99757971, 0.99964525, 0.96656999,
        0.99882205, 0.99913428],
       [0.99320691, 0.99591978, 0.99581111, 0.995426

Main antiga

In [131]:
#Main
feature_engineering_method = []
algoritmo=[]
accuracy = []
precision = []
f1_score = []
recall = []

def main():
    df_business_filadelfia,df_review_filadelfia,df_user_filadelfia = load_data()

    df_business_filadelfia=df_business_filadelfia.sample(1000)
    df_review_filadelfia=df_review_filadelfia.sample(1000)
    df_user_filadelfia=df_user_filadelfia.sample(1000)

    for a in methods_pre_processing:
        df_review_filadelfia_pre_processado = pre_processing(df_review_filadelfia,a)
        # print(df_business_filadelfia)
        # print(df_review_filadelfia)
        # print(df_user_filadelfia)

        for b in methods_feature_engineering:
            feature_engineering_method.append(b)
            final_data = feature_engineering(df_review_filadelfia_pre_processado,b)
            # print(final_data)
            for c in add_features_decision:
                # print(a,b,c)
                if c == 'yes':
                    # business_data,users_data = add_features(final_data,df_business_filadelfia,df_review_filadelfia,df_user_filadelfia) #adicionamos as repetivas features a cada matriz
                    colunas = add_features(final_data,df_business_filadelfia,df_review_filadelfia,df_user_filadelfia) #adicionamos as repetivas features a cada matriz
                    print(colunas)
                # else:
                #     business_data=final_data
                #     users_data=final_data
                
                # general_trainset, general_testset,user_trainset, user_testset,business_trainset, business_testset = split_data(final_data,business_data,users_data)

                # print('general_trainset')
                # print(general_trainset)

                # print('general_testset')
                # print(general_testset)

                # print('user_trainset')
                # print(user_trainset)

                # print('user_testset')
                # print(user_testset)

                # print('business_trainset')
                # print(business_trainset)

                # print('business_testset')
                # print(business_testset)

    #             for d in algorithms:
    #                 algoritmo.append(d)

    #                 if d == 'CF-UB':
    #                     algo = KNNBasic(sim_options={'user_based': True})
    #                     algo.fit(user_trainset)
    #                     predictions = algo.test(user_testset)

    #                 elif d == 'CF-IB':
    #                     algo = KNNBasic(sim_options={'user_based': False})
    #                     algo.fit(business_trainset)
    #                     predictions = algo.test(business_testset)

    #                 elif d == 'UBH':
    #                     similarity_matrix = cosine_similarity(user_trainset)
    #                     similarity_matrix = pd.DataFrame(similarity_matrix, index=user_trainset.index, columns=user_trainset.index)

    #                     for user_id in user_testset['user_id']:
    #                         similar_users = get_top_n_similar_users(user_id, n=5)
    #                         recommended_restaurants = get_recommended_restaurants(user_id, similar_users, n_restaurants=5)
    #                         best_restaurant = recommended_restaurants.idxmax()


    #                 elif d == 'IBH':
    #                     for business_id in business_testset['business_id']:
    #                         similarity_matrix = cosine_similarity(business_trainset)
    #                         similarity_matrix = pd.DataFrame(similarity_matrix, index=business_trainset.index, columns=business_trainset.index)

    #                         recommendations = recommend_for_user(user_id, philly_restaurants, algo, similarity_matrix, n=5)


    #                 elif d == 'UIBH':
    #                     similarity_matrix = cosine_similarity(user_trainset,business_trainset)
    #                     similarity_matrix = pd.DataFrame(similarity_matrix, index=user_trainset.index, columns=business_trainset.index)


In [ ]:
def recommend_similar_restaurants(restaurant_id, philly_restaurants, similarity_matrix, n=5):
    # Obter o índice do restaurante
    idx = philly_restaurants.index[philly_restaurants['business_id'] == restaurant_id].tolist()[0]
    
    # Obter similaridade do restaurante com todos os outros
    similars_indices = similarity_matrix[idx].argsort()[::-1]  # Do mais similar para o menos similar
    
    # Excluir o próprio restaurante da recomendação
    similars_indices = similars_indices[similars_indices != idx]
    
    # Selecionar os n mais similares
    similars_restaurants = philly_restaurants.iloc[similars_indices[:n]]
    
    return similars_restaurants[['business_id', 'name', 'categories', 'stars']]

In [ ]:
# Função para recomendar com base nos restaurantes que o usuário já avaliou bem
def recommend_for_user(user_id, philly_restaurants, algo, similarity_matrix, n=5):
    # Obter os restaurantes avaliados pelo usuário
    user_reviews = ratings[ratings['user_id'] == user_id]
    highly_rated = user_reviews[user_reviews['stars'] >= 4]['business_id']
    
    # Para cada restaurante que o usuário gostou, recomendar restaurantes similares
    recommendations = pd.DataFrame()
    # print('highly rated ',highly_rated)

    for restaurant_id in highly_rated:
        try:
            # Obter o índice do restaurante
            inner_id = algo.trainset.to_inner_iid(restaurant_id)
            
            # Obter os restaurantes mais similares usando o modelo treinado
            neighbors = algo.get_neighbors(inner_id, k=n)
            # print('neighbors ',neighbors)
            # Converter os índices internos para IDs de restaurantes
            similar_restaurant_ids_knn = [algo.trainset.to_raw_iid(inner_id) for inner_id in neighbors]
            
            # Obter os detalhes dos restaurantes similares usando o modelo treinado
            similar_restaurants_knn = philly_restaurants[philly_restaurants['business_id'].isin(similar_restaurant_ids_knn)]
            # print('similar_restaurants_knn ',similar_restaurants_knn)
        except ValueError:
            # Se o restaurante não estiver no conjunto de treino, retornar um DataFrame vazio
            similar_restaurants_knn = pd.DataFrame()
        
        # Obter os detalhes dos restaurantes similares usando a matriz de similaridade
        similar_restaurants_matrix = recommend_similar_restaurants(restaurant_id, philly_restaurants, similarity_matrix, n)
        # print('similar_restaurants_matrix ',similar_restaurants_matrix)
        # Combinar as recomendações de ambos os métodos
        combined_recommendations = pd.concat([similar_restaurants_knn, similar_restaurants_matrix]).drop_duplicates(subset='business_id')
        
        recommendations = pd.concat([recommendations, combined_recommendations])
        # print('recommendations ',recommendations)
    # Remover duplicatas e ordenar por popularidade (opcional: você pode melhorar o critério de ordenação)
    recommendations = recommendations.drop_duplicates(subset='name').sort_values(by='stars', ascending=False)
    # print('recommendations ',recommendations)
    return recommendations['name'].head(n)



In [12]:
def get_top_n_similar_users(user_id, n,similarity_matrix,user_trainset):
    if user_id in similarity_matrix.index:
        similar_users = similarity_matrix[user_id].sort_values(ascending=False).index[1:n+1]
    else:
        from sklearn.neighbors import NearestNeighbors
        knn = NearestNeighbors(n_neighbors=n, metric='cosine')
        knn.fit(user_trainset)
        distances, indices = knn.kneighbors(user_trainset.loc[user_id].values.reshape(1, -1), n_neighbors=n+1)
        similar_users = user_trainset.index[indices.flatten()][1:]
    return similar_users


In [13]:
def get_recommended_restaurants(user_id, similar_users, n,user_trainset):
    target_user_ratings = user_trainset.loc[user_id]
    target_user_visited = target_user_ratings[target_user_ratings > 0].index

    similar_users_ratings = user_trainset.loc[similar_users]
    similar_users_ratings = similar_users_ratings.drop(columns=target_user_visited, errors='ignore')

    top_rated_restaurants = similar_users_ratings.mean().sort_values(ascending=False).head(n)
    return top_rated_restaurants
